In [3]:
import sqlite3
import json
from datetime import datetime



In [15]:
######################
#TUTORIAL code-block 1
######################

timeframe = '2005-12'
#build one transaction and connect data all at once
sql_transaction = []

connection = sqlite3.connect('{}.db'.format(timeframe))
#cursor
c = connection.cursor()

def create_table() :
    c.execute("""CREATE TABLE IF NOT EXISTS parent_reply
    (parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, 
    parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)""")

def format_data(data):
    data = data.replace("\n", " newlinechar ").replace("\r", " newlinechar ").replace('"', "'")
    return data

def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]' or data == '[removed]':
        return False
    else:
        return True
    


def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0] # 0 cause we are only selecting 1 commenting
        else:
            return False
    except Exception as e:
        #print("Find Parent)
        return False
    

In [16]:
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0] # 0 cause we are only selecting 1 commenting
        else:
            return False
    except Exception as e:
        #print("Find Parent)
        return False 

In [ ]:
#parent_data = find_parent(parent_id)    
if __name__ == "__main__":
        create_table()
        #how many rows we have created
        row_counter = 0
        #how many parents and child pairs we found
        paired_rows = 0
        
        with open("RC_2005-12".format(timeframe.split('-')[0], timeframe), buffering = 1000) as f:
            for row in f:
                print(row)
                row_counter += 1
                row = json.loads(row)
                parent_id = row['parent_id']
                body = format_data(row['body'])
                created_utc = row['created_utc']
                score = row['score']
                subreddit = row['subreddit']
                parent_data = find_parent(parent_id)
                
                #since there is a lot of useless comments on reddit,
                #we only keep certain comments
                if score >= 2: #means somebody saw this comment at least 
                    #now we check to see if there's a comment with this parent id with a better score
                    existing_comment_score = find_existing_score(pid)
                    if existing_comment_score:
                        if score > existing_comment_score:
                            
                            
                    
                    

In [ ]:
######################
#TUTORIAL code-block 1
######################


